In [892]:
from random import randint, shuffle, random
import numpy as np

# Matrice d'adjacence

In [893]:
# Chargement de la matrice adjacence à partir d'un fichier
def load_matrix():
    mat = []
    with open('mat_adjacence', 'r') as file:   
        for l in file.readlines():  
            val = [int(x) for x in l.split()]
            mat.append(val)            
    return mat

# Créer une matrice d'adjacence dont le cout du circuit Hamiltonien optimal est celui de la périphérie  
def create_matrix(n_ville):
    mat = np.zeros((n_ville, n_ville))
    
    for i in range(n_ville):
        for j in range(i, n_ville):
            cout = randint(1, 10)
            if i == j:
                cout = 0
            i_after = i+1 if i+1 < n_ville else 0
            if i_after == j:
                cout = 1
                 
            mat[i][j] = cout
            mat[j][i] = cout
    mat[n_ville-1][0] = 1
    mat[0][n_ville-1] = 1

    return mat

In [894]:
def population_init(nb_ville, size):
    population = []
    
    for _ in range(size):
        circuit = list(range(nb_ville))
        shuffle(circuit) # circuit random
        population.append([circuit, 5])
        
    return population

def get_path_cost(cost_mat, path): # Renvoie le coût d'un chemin, par rapport à la matrice d'adjacence
    travel_length = 0
    
    for i in range(0, len(path)):
        depart = path[i-1] # On compte aussi l'arrète entre la premiere et dernière ville (-1 en python -> dernier élément) 
        arrival = path[i]
        travel_length += cost_mat[depart][arrival]
    
    return travel_length  



# Agent

In [895]:

class Agent:
    def __init__(self, nodes_to_explore, deposit_Q, prob_explore_Y=1, pheromone_influence=1, distance_influence=1):
        self.starting_node = 0 # Toute les fourmies commencent au même endroit
        self.deposit_Q = deposit_Q
        self.null_prob = prob_explore_Y 
        # Pour la decision de l'exploration
        self.pheromone_influence = pheromone_influence # Importance des phéromone
        self.distance_influence = distance_influence # Importance de la distance d'une ville
        self.reset_exploration(list(nodes_to_explore)) 
    
     
    def reset_exploration(self, nodes_to_explore):
        self.unexplored = nodes_to_explore
        self.unexplored.remove(self.starting_node)
        self.explored = [self.starting_node]
    
    
    def deposite_pheromone(self, mat_cost, mat_pheromone):
        pheromone_quantity = self.deposit_Q / get_path_cost(mat_cost, self.explored) 
        
        for i in range(1, len(self.explored)):
            depart = self.explored[i-1]
            arrival = self.explored[i]
            
            # On dépose les phéromones sur les arrettes
            mat_pheromone[depart][arrival] += pheromone_quantity
          
          
      
    def choose_next_location(self, mat_cost, mat_pheromone):
        # On choisit la prochaine ville à explorer aléatoirement selon les phéromones,
        # avec une probabilité non-nul d'explorer une ville qui n'a pas reçu de dépots de phéromone.
        # Retourne un indice parmis la liste des unexplored
        
        depart = self.explored[-1] 
        sum = 0
        
        # On somme les differente pheromone pour pondérer les probabilités
        for arrival in self.unexplored:
            # On a une proba non nul pour qu'un agent puisse explorer une ville jamais exploré par les autres agents controllé par le parametre Y
            sum += self.null_prob \
                + mat_pheromone[depart][arrival] ** self.pheromone_influence \
                + 1/mat_cost[depart][arrival] ** self.distance_influence # ηij la visibilité, qui est égale à l’inverse de la distance de deux villes i et j 
        
        # random qui decide de la prochaine ville à explorer
        r = random() *  sum
        s = 0 
        # Recherche de la ville qui vient d'être choisit
        for i, arrival in enumerate(self.unexplored): 
            s += self.null_prob \
                + mat_pheromone[depart][arrival] ** self.pheromone_influence \
                + 1/mat_cost[depart][arrival] ** self.distance_influence
            
            # On s'arrette lorsque l'on a trouvé la prochaine ville, et on retourne son indice dans unexplored
            if s >= r:
                return i
            
    def explore(self, mat_cost, mat_pheromone):
        # Tant qu'on a pas tout exploré
        while len(self.unexplored) > 0:
            # On choisit la prochaine destination parmis ceux qui reste à explorer 
            next_i = self.choose_next_location(mat_cost, mat_pheromone) 
            curr_node = self.unexplored.pop(next_i)
            self.explored.append(curr_node)
            
        # Une fois la tournée terminé, l'agent dépose les phéromones
        self.deposite_pheromone(mat_cost, mat_pheromone)
        # On reset les villes explorer pour pouvoir recommencer l'exploration 
        self.reset_exploration(self.explored)
                
            

In [896]:
def evaporation(mat_pheromone, disipation_rate):
    for i in range(len(mat_pheromone)):
        for j in range(len(mat_pheromone[0])):
            if mat_pheromone[i][j] > 0:
                mat_pheromone[i][j] -= disipation_rate
                
def index_max(arr, exclude_indices): # retourne l'indice dont la valeur est maximal, et qui n'est pas dans exclude_indices   
    # On n'explore pas a partir du premier, au cas où l'élément 0 fait partir de exclude_indices
    best_i = None 
    for i in range(len(arr)): 
        if best_i == None:
            if i not in exclude_indices: 
                best_i = i
            continue 
        
        if arr[best_i] < arr[i] and i not in exclude_indices:
            best_i = i
            
    return best_i

def find_best(mat_pheromone): # Trouve le chemin le plus parcourus
    path = [0] # On démart à 0, car on cherche un cycle hamiltonien, donc le meilleur circuit passera forcement par cette ville, indépendament de l'ordre.
    # Pour chaque ville
    for _ in range(1, len(mat_pheromone)):
        depart = path[-1]
        arrival = index_max(mat_pheromone[depart], path)
        path.append(arrival)
    return path

# Programme test

In [897]:

def main():
    size = 10
    
    epoch_max = 100
    agents_number = 10 
    disipation_rate = 1
    deposit_Q = 1
    prob_explore_Y = 1
    pheromone_influence = 5
    distance_influence = 2
    
    mat_cost = create_matrix(size)
    print("matrice d'adjacence")
    print(mat_cost)
    mat_pheromone = np.zeros((size, size))
    agents = [Agent(list(range(size)), deposit_Q, prob_explore_Y, pheromone_influence, distance_influence) for _ in range(agents_number)]
    
    disipation_rate = 0
    
    for i in range(epoch_max):
        # On avance chaque agents dans leurs exploration
        for agent in agents:
            agent.explore(mat_cost, mat_pheromone) # Un agent avance d'un noeud
        # On évapore les phéromones a chaque fin d'itération
        evaporation(mat_pheromone, disipation_rate)
        print("iteration: ", i, ", meilleurs chemins:", get_path_cost(mat_cost, find_best(mat_pheromone)) )
    
    best_path = find_best(mat_pheromone) 
    print("Meilleur chemin: ", best_path)
    print("Cout: ", get_path_cost(mat_cost, best_path))
    
main()

matrice d'adjacence
[[ 0.  1.  1.  9.  2.  5.  1.  8.  6.  1.]
 [ 1.  0.  1.  5.  7.  4.  5.  7.  2.  8.]
 [ 1.  1.  0.  1.  3.  1.  9.  3.  7.  2.]
 [ 9.  5.  1.  0.  1.  1.  1. 10.  5.  8.]
 [ 2.  7.  3.  1.  0.  1.  3.  8.  7.  2.]
 [ 5.  4.  1.  1.  1.  0.  1.  5.  1.  3.]
 [ 1.  5.  9.  1.  3.  1.  0.  1.  8.  2.]
 [ 8.  7.  3. 10.  8.  5.  1.  0.  1.  2.]
 [ 6.  2.  7.  5.  7.  1.  8.  1.  0.  1.]
 [ 1.  8.  2.  8.  2.  3.  2.  2.  1.  0.]]
iteration:  0 , meilleurs chemins: 21.0
iteration:  1 , meilleurs chemins: 16.0
iteration:  2 , meilleurs chemins: 16.0
iteration:  3 , meilleurs chemins: 13.0
iteration:  4 , meilleurs chemins: 21.0
iteration:  5 , meilleurs chemins: 13.0
iteration:  6 , meilleurs chemins: 17.0
iteration:  7 , meilleurs chemins: 10.0
iteration:  8 , meilleurs chemins: 10.0
iteration:  9 , meilleurs chemins: 10.0
iteration:  10 , meilleurs chemins: 10.0
iteration:  11 , meilleurs chemins: 10.0
iteration:  12 , meilleurs chemins: 10.0
iteration:  13 , meilleurs